In [1]:
%load_ext autoreload
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import torch
from torch import nn

from mvn.utils.minimon import MiniMon
from mvn.utils.misc import flush_cache

from mvn.ipynb import get_args, get_config, build_labels
from mvn.pipeline.setup import setup_dataloaders, setup_experiment, build_env
from mvn.pipeline.training import do_train

In [3]:
flush_cache()

is_distributed = False
master = True
device = torch.device(0)
print('using dev {}'.format(device))
    
args = get_args()

# just to get a feeling of the dataset
# labels, mask, indices = build_labels(config.dataset.train.labels_path, 10000)
# labels, mask, indices = build_labels(config.dataset.train.labels_path, 10, allowed_subjects=['S9', 'S11'])

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |       0 B  |       0 B  |       0 B  |       0 B  |
|       from large pool |       0 B  |       0 B  |       0 B  |       0 B  |
|       from small pool |       0 B  |       0 B  |       0 B  |       0 B  |
|---------------------------------------------------------------------------|
| Active memory         |       0 B  |       0 B  |       0 B  |       0 B  |
|       from large pool |       0 B  |       0 B  |       0 B  |

In [ ]:
config = get_config(args)
do_train(None, None, config, device, is_distributed, master)

cam2cam model (shared):
                      backbone has      10896 params (~  0.3) %
                    R_backbone has    1775110 params (~ 49.9) %
                    t_backbone has    1774339 params (~ 49.8) %
total params:              3560345
cam2cam estimation => adding 3560345 params to grad ...
  training dataset length: 12
  validation dataset length: 5
Experiment name: human36m_alg_AlgebraicTriangulationNet@17.05.2021-15:04:36
#[do_train]: epoch    0 has started!
#[cam2cam]: training batch iter 0 losses: GEO ~ 9.117, TRANS ~ 201.404, POSE ~ 82228.844, ROTO ~ 7.730, 3D ~ 637.620, SELF R ~ 40.835, SELF t ~ 45.786, TOTAL ~ 1137.540
#[cam2cam]: training batch iter 0 MPJPE: ~ 412.240 mm
#[cam2cam]: training batch iter 1 losses: GEO ~ 7.184, TRANS ~ 201.310, POSE ~ 85811.969, ROTO ~ 7.464, 3D ~ 636.219, SELF R ~ 47.035, SELF t ~ 33.083, TOTAL ~ 1172.730
#[cam2cam]: training batch iter 1 MPJPE: ~ 544.900 mm
#[cam2cam]: training batch iter 2 losses: GEO ~ 7.502, TRANS ~ 199.868, P